In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import cv2
import numpy as np

# **导入必须的库**
- `cv2`、`numpy`、`transforms`用来对图像数据进行预处理；
- `Dataset`, `DataLoader`用来加载数据；
- `nn`用来定义一个轻量的`CNN`模型；
- `optim`用来训练时优化模型的参数。

In [ ]:
def remove_black_lines(image_path):
    # 读取图像
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("无法加载图像，请检查路径是否正确！")

    # 检查是否为灰度图像
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    # 创建黑色像素掩膜（RGB都小于50）
    threshold = 50
    mask = np.all(img < [threshold, threshold, threshold], axis=2).astype(np.uint8) * 255

    # 如果掩膜为空，直接返回原图
    if np.any(mask):
        # 使用图像修复去除干扰线
        result = cv2.inpaint(img, mask, inpaintRadius=2, flags=cv2.INPAINT_TELEA)
    else:
        result = img.copy()

    return result

# 经观察，验证码的干扰线均为**黑色**，而正常的字符基本没有黑色，因此预处理先去掉黑色干扰线。
# 1. **读取图像：**  
   - 使用 `OpenCV` 的 `cv2.imread` 函数读取图像。默认情况下，图像以 `BGR` 格式加载。  
---
# 2. **创建黑色像素掩膜：**  
   - `np.all(img < [50, 50, 50], axis=2)`：检查每个像素的 `RGB` 值是否都小于 50（接近黑色）。`axis=2` 表示对每个像素的三个通道进行检查。
   - `astype(np.uint8) * 255`：将布尔掩膜转换为 8 位整数格式（0 或 255），以便用作 `OpenCV` 的掩膜，黑色则返回255（白色），否则返回0，不作修改。
   - 注：掩膜通常是一个与原始图像大小相同的二值或布尔图像，其中，选定的区域被标记为`1`（或`True`），而其余区域被标记为`0`（或`False`）。使用掩膜时，只处理掩膜数组中值为`1`（或`True`）的对应像素，而忽略值为`0`（或`False`）的像素。
---  
# 3. **使用图像修复去除干扰线：**  
   - `cv2.inpaint`：使用 `OpenCV `的图像修复功能，通过掩膜指定需要修复的区域。
   - `inpaintRadius=2`：修复的半径，可以根据需要调整。
   - `flags=cv2.INPAINT_TELEA`：使用 `Telea` 算法进行修复，适合处理黑色线条等简单干扰。

In [ ]:
# 1. 数据集类定义
class CaptchaDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.filenames = [f for f in os.listdir(data_dir) if f.endswith(('.jpg','.png'))]
        
        # 创建字符到索引的映射
        self.char2idx = {chr(ord('a')+i): i for i in range(26)}
        for i in range(10):
            self.char2idx[str(i)] = i + 26

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.filenames[idx])
        img = remove_black_lines(img_path)
        
        # 转换张量
        if self.transform:
            img = self.transform(img)
        
        # 从文件名获取标签
        label_str = self.filenames[idx].split('_')[0]
        label = [self.char2idx[c] for c in label_str]
        return img, torch.tensor(label)

# 1. **初始化方法** (`__init__`)
   - `data_dir`：验证码图像的存储目录。
   - `transform`：用于图像预处理的转换函数。
   - `self.filenames`：筛选出目录中所有以 `.jpg` 或 `.png` 结尾的文件，文件名的格式需要是 `label_xxxx.jpg`，其中 `label` 是验证码的文本内容，且必须符合字符集（小写字母和数字）。
   - `self.char2idx`：创建了一个字符到索引的映射，由小写字母（`a-z`）和数字（`0-9`）组成，对应索引`0—35`共36个。
   - **注：**
     - `ord()`将字符串表示的字符转换为对应的整数，适用于`Unicode`字符，如`ord('a')`，输出`97`；`ord('A')`，输出`65`；
     - `chr()`将一个范围是`range(256)（0~255）`内的整数作为参数，返回一个对应的字符，如`chr(97)`，输出`a`。  
---
# 2. `__len__ `**方法**
   - 返回数据集中的图像数量，即文件总数。  
---
# 3. `__getitem__` **方法**

   **(1). 读取图像：**
      - 使用 `os.path.join` 构造完整的图像路径。
      - 调用 `remove_black_lines` 函数去除图像中的黑色线条。

   **(2). 应用转换：**
      - 如果提供了 `transform`，则对图像进行应用预处理转换（如 ToTensor、归一化等）。

   **(3). 获取标签：**
      - 从文件名中提取标签字符串（假设文件名格式为 `label_filename.jpg`）。
      - 将标签字符串转换为索引列表（基于 `char2idx` 映射）。如`label`为`'abcd'`，则返回索引列表`[0,1,2,3]`。

   **(4). 返回结果：**
      - 返回处理后的图像（shape是`[3,32,90]`）和对应的标签张量（shape是`[1,4]`）。


In [ ]:
# 2. 轻量CNN模型（<1MB）
class TinyCNN(nn.Module):
    def __init__(self, num_chars=4, num_classes=36):
        super(TinyCNN, self).__init__()
        self.features = nn.Sequential(
            # 输入: 3x32x90
            nn.Conv2d(3, 8, 3, padding=1),  # 16x32x90
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16x45
            
            nn.Conv2d(8, 16, 3, padding=1),  # 32x16x45
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x8x22
            
            nn.Conv2d(16, 32, 3, padding=1),  # 64x8x22
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 11))  # 64x4x11
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(32*4*11, 128),
            nn.Dropout(0.3),
            nn.LayerNorm(128),  # 🌟 层归一化
            nn.Linear(128, num_chars*num_classes)
        )
        self.num_chars = num_chars
        self.num_classes = num_classes

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x.view(-1, self.num_chars, self.num_classes)

# **模型结构详解**
# 1. **特征提取部分** (`self.features`)
```py
self.features = nn.Sequential(
    # 输入: 3x32x90
    nn.Conv2d(3, 8, 3, padding=1),  # 16x32x90
    nn.BatchNorm2d(8),
    nn.ReLU(),
    nn.MaxPool2d(2),  # 16x16x45

    nn.Conv2d(8, 16, 3, padding=1),  # 32x16x45
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(2),  # 32x8x22

    nn.Conv2d(16, 32, 3, padding=1),  # 64x8x22
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((4, 11))  # 64x4x11
)
```
- `self.features` 是一个 `nn.Sequential` 容器，包含卷积层、批量归一化、激活函数和池化层。它的作用是从输入图像中提取特征。

- **输入：** 假设输入图像的大小为 3x32x90（3 个通道，高为 32，宽为 90）。
---
- **第一层卷积：**

  - `nn.Conv2d(3, 8, 3, padding=1)`：输入通道为 3，输出通道为 8，卷积核大小为 3x3，填充为 1。

  输出大小为 **8x32x90**。

  - 批量归一化：`nn.BatchNorm2d(8)`：对 8 个通道进行批量归一化。

  - ReLU 激活函数：`nn.ReLU()`：将所有负值置为 0，保留非线性特性。

  - 最大池化：`nn.MaxPool2d(2)`：对特征图进行 2x2 的最大池化，步长为 2。

  输出大小为 **8x16x45**。
---
- **第二层卷积：**

  - `nn.Conv2d(8, 16, 3, padding=1)`：输入通道为 8，输出通道为 16，卷积核大小为 3x3，填充为 1。

  输出大小为 **16x16x45**。

  - 批量归一化：`nn.BatchNorm2d(16)`：对 16 个通道进行批量归一化。

  - ReLU 激活函数：`nn.ReLU()`：非线性激活。

  - 最大池化：`nn.MaxPool2d(2)`：对特征图进行 2x2 的最大池化，步长为 2。

  输出大小为 **16x8x22**。
---
- **第三层卷积：**

  - `nn.Conv2d(16, 32, 3, padding=1)`：输入通道为 16，输出通道为 32，卷积核大小为 3x3，填充为 1。

  输出大小为 **32x8x22**。

  - 批量归一化：`nn.BatchNorm2d(32)`：对 32 个通道进行批量归一化。

  - ReLU 激活函数：`nn.ReLU()`：非线性激活。

  - 自适应平均池化：`nn.AdaptiveAvgPool2d((4, 11))`：将特征图的大小调整为 4x11，无论输入大小如何。

  输出大小为 **32x4x11**。  

---


# 2. **分类器部分** (`self.classifier`)

- `self.classifier` 是一个全连接网络，用于将特征图映射到最终的类别预测。
```py
self.classifier = nn.Sequential(
    nn.Linear(32*4*11, 128),
    nn.Dropout(0.3),
    nn.LayerNorm(128),  # 层归一化
    nn.Linear(128, num_chars*num_classes)
)
```
- **输入：** 特征提取部分的输出大小为 **32x4x11**，因此展平后的输入大小为 **32x4x11 = 1408**。

- **第一层全连接：**

  - `nn.Linear(32*4*11, 128)`：将 1408 维特征映射到 128 维。

- **Dropout**：`nn.Dropout(0.3)`：随机丢弃 30% 的神经元，防止过拟合。

- **层归一化**：`nn.LayerNorm(128)`：对 128 维特征进行层归一化。

- **第二层全连接：**

  - `nn.Linear(128, num_chars*num_classes)`：将 128 维特征映射到 `num_chars*num_classes` 维。

  - 假设 `num_chars=4`（验证码长度为 4），`num_classes=36`（每个字符有 36 个类别），则输出大小为 **4x36 = 144**。
---
# 3. **前向传播** (`forward`)
```
def forward(self, x):
    x = self.features(x)  # 提取特征
    x = torch.flatten(x, 1)  # 展平特征图
    x = self.classifier(x)  # 全连接层
    return x.view(-1, self.num_chars, self.num_classes)# 重塑输出
```
   - **特征提取**：`x = self.features(x)`。

   - **展平**：`torch.flatten(x, 1)` 将特征图展平为二维张量。

   - **分类器**：`x = self.classifier(x)`。

   - **重塑输出**：`x.view(-1, self.num_chars, self.num_classes)` 将输出重塑为 `(batch_size, num_chars, num_classes)` 的形状。例如，对于一个批量大小为 32 的输入，输出形状为 `(32, 4, 36)`。

---

# **参数计算**

# 1. **特征提取部分**


|  | 第一层卷积 | 批量归一化 | 第二层卷积 | 批量归一化 | 第三层卷积 | 批量归一化 |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 输入通道 | 3 | - | 8 | - | 16 | - |
| 输出通道 | 8 | - | 16 | - | 32 | - |
| 卷积核大小 | 3x3 | - | 3x3 | - | 3x3 | - |
| 参数数量 | 3×3×3×8+8=224（包括偏置项） | 2×8=16（均值和方差） | 3×3×8×16+16=1168 | 2×16=32 | 3×3×16×32+32=4640 | 2×32=64 |
|  特征提取部分总参数 |   |    |  224+16+1168+32+4640+64=6144  |    |    |    |


---

# 2. **分类器部分**

|  | **第一层全连接**              | **层归一化**  | **第二层全连接**        |
|:----:|:-----------------------:|:---------:|:-----------------:|
| 输入大小 | 32x4x11=1408            | -         | 128               |
| 输出大小 | 128                     | -         | 4×36=144          |
| 参数数量 | 1408×128+128=180352     | 2×128=256 | 128×144+144=18624 |
|  分类器部分总参数  |   |180352+256+18624=199232           |                   |

---

# 3. **总参数数量**

- **总参数数量**：`6144+199232=205376`。

---

# **总结**

  - **模型结构**：`TinyCNN `是一个轻量级的卷积神经网络，适用于验证码识别任务。

  - **特征提取部分**：通过卷积层、批量归一化和池化层逐步提取图像特征。

  - **分类器部分**：将提取的特征映射到最终的类别预测。

  - **总参数数量**：`205,376` 个参数，模型预计大小为`205376 × 4字节 = 821504字节`，约为`802.6 KB`(因为1 KB=1024 字节)，模型较为轻量级。

In [1]:
# 4. 训练循环
def train(model, train_dataloader, val_dataloader, epochs=300, save_threshold=0.9985):
    model.train()  # 设置模型为训练模式
    criterion = nn.CrossEntropyLoss()  # 定义损失函数
    optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)  # 定义优化器
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=3e-3, steps_per_epoch=len(train_dataloader), epochs=epochs
    )  # 定义学习率调度器

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for inputs, labels in train_dataloader:
            optimizer.zero_grad()  # 清空梯度
            outputs = model(inputs)  # 前向传播

            # 计算四个字符的损失
            loss = sum(criterion(outputs[:, i, :], labels[:, i]) for i in range(4))
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
            scheduler.step()  # 更新学习率

            # 计算准确率
            _, predicted = torch.max(outputs, 2)
            correct += (predicted == labels).all(1).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

        # 打印训练信息
        train_loss = total_loss / total
        train_acc = correct / total
        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Train Acc: {train_acc:.4%}, LR: {optimizer.param_groups[0]['lr']:.6f}")

        # 验证模型
        val_loss, val_acc = validate(model, val_dataloader, criterion)
        print(f"Epoch {epoch + 1}/{epochs}, Val Loss: {val_loss:.6f}, Val Acc: {val_acc:.4%}")

        # 保存模型
        if val_acc > save_threshold:
            torch.save(model.state_dict(), f"{val_acc:.6f}_light_captcha_model.pth")
            print(f"Model saved with validation accuracy: {val_acc:.4%}")


# 5. 验证函数
def validate(model, dataloader, criterion):
    model.eval()  # 设置模型为评估模式
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # 关闭梯度计算
        for inputs, labels in dataloader:
            outputs = model(inputs)  # 前向传播

            # 计算损失
            loss = sum(criterion(outputs[:, i, :], labels[:, i]) for i in range(4))
            total_loss += loss.item()

            # 计算准确率
            _, predicted = torch.max(outputs, 2)
            correct += (predicted == labels).all(1).sum().item()
            total += labels.size(0)

    val_loss = total_loss / total
    val_acc = correct / total
    return val_loss, val_acc


# 1. **训练函数** (`train`)
训练函数的作用是完成模型的训练过程，包括前向传播、计算损失、反向传播、更新参数以及学习率调度。
   - `model`：要训练的模型。
   - `train_dataloader`：训练数据加载器。
   - `val_dataloader`：验证数据加载器。
   - `epochs`：训练的总轮数，默认为 300。
   - `save_threshold`：验证准确率超过该阈值时保存模型，默认为 0.9985。
---
# 2. **损失函数和优化器**

   - `criterion`：使用交叉熵损失函数（`CrossEntropyLoss`），适用于多分类问题。
   - `optimizer`：使用 `AdamW` 优化器，学习率（`lr`）为 3e-4，权重衰减（`weight_decay`）为 1e-4，用于正则化。
---
# 3. **学习率调度器**
```
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=3e-3, steps_per_epoch=len(train_dataloader), epochs=epochs)
```

   - `OneCycleLR`：一种学习率调度器，会在一个训练周期内先增加学习率，再降低学习率。
   - `max_lr`：最大学习率，设置为 3e-3。
   - `steps_per_epoch`：每个 epoch 的迭代次数，等于训练数据加载器的长度。
   - `epochs`：总训练轮数。
---
# 4. **训练过程**

   - 清空梯度：`optimizer.zero_grad()`，防止梯度累积。
   - 前向传播：`outputs = model(inputs)`，模型对输入数据进行预测。
---
# 5. **计算损失**
```py
loss = sum(criterion(outputs[:, i, :], labels[:, i]) for i in range(4))
```

- **损失计算**：验证码有 4 个字符，每个字符的预测结果是一个独立的分类问题。因此，对每个字符分别计算损失，并将它们相加。

---

**(1).  背景**
>- `outputs`：模型的预测输出，形状为 `(batch_size, num_chars, num_classes)`。
>- `batch_size`：批量大小。
>- `num_chars`：验证码中的字符数量（这里是 4）。
>- `num_classes`：每个字符的类别数量（这里是 36）。指一个字符有36种可能的类别。
>- `labels`：真实标签，形状为 `(batch_size, num_chars)`。每个标签是一个整数，表示字符的类别索引。

**(2). 损失函数**
使用 `nn.CrossEntropyLoss` 作为损失函数。`CrossEntropyLoss` 的输入要求是：
   - **预测值**：形状为 `(batch_size, num_classes)`，表示每个样本的类别概率分布。
   - **真实标签**：形状为 `(batch_size,)`，表示每个样本的真实类别索引。
  
>`nn.CrossEntropyLoss`的计算公式为：`nn.logSoftmax()`和`nn.NLLLoss()`的整合版本：

![]( 搜狗截图20250216054247.png "")

**(3). 逐字符计算损失**
>- `:`表示提取所有
>- `outputs[:, i, :]`：提取第 `i`个字符的预测概率分布，形状为 `(batch_size, num_classes)`。
   > 例如，`outputs[:, 0, :]` 是所有样本的第一个字符的预测概率分布。
>- `labels[:, i]`：提取第 `i` 个字符的真实标签，形状为 `(batch_size,)`。
   > 例如，`labels[:, 0]` 是所有样本的第一个字符的真实标签。
---
# **示例解释**
假设我们有一个批量大小为 2 的验证码数据，每个验证码有 4 个字符，每个字符有 36 个类别。模型的输出和真实标签如下：

- `outputs` 的形状为 `(2, 4, 36)`，表示每个样本的每个字符的类别概率分布:
  >这里概率可以大于1，因为`nn.CrossEntropyLoss`会自动对输入模型的预测值进行`softmax`。因此在多分类问题中，如果使用`nn.CrossEntropyLoss()`，则预测模型的输出层无需添加softmax层

```py
outputs = torch.tensor([
    [
        [0.1, 0.2, 0.7, 0.0, ...],  # 第一个样本的第一个字符的概率分布
        [0.8, 0.1, 0.05, 0.05, ...],  # 第一个样本的第二个字符的概率分布
        [0.01, 0.01, 0.95, 0.03, ...],  # 第一个样本的第三个字符的概率分布
        [0.2, 0.3, 0.4, 0.1, ...]  # 第一个样本的第四个字符的概率分布
    ],
    [
        [0.05, 0.05, 0.9, 0.0, ...],  # 第二个样本的第一个字符的概率分布
        [0.1, 0.7, 0.1, 0.1, ...],  # 第二个样本的第二个字符的概率分布
        [0.02, 0.02, 0.96, 0.0, ...],  # 第二个样本的第三个字符的概率分布
        [0.1, 0.2, 0.6, 0.1, ...]  # 第二个样本的第四个字符的概率分布
    ]
])
```
- `labels` 的形状为 `(2, 4)`，表示每个样本的每个字符的真实类别索引：

```py
labels = torch.tensor([
    [2, 0, 2, 2],  # 第一个样本的字符标签
    [2, 1, 2, 2]   # 第二个样本的字符标签
])
```

**下面计算损失：**

**第一个字符的损失：**
```py
criterion(outputs[:, 0, :], labels[:, 0])
```

- `outputs[:, 0, :]`：

```py
torch.tensor([
    [0.1, 0.2, 0.7, 0.0, ...],  # 第一个样本的第一个字符
    [0.05, 0.05, 0.9, 0.0, ...]  # 第二个样本的第一个字符
])
```
- `labels[:, 0]`：

```py
torch.tensor([2, 2]) #2个样本第一个字符的标签
```

第一个字符的损失值为这两个样本的平均损失：
$loss1=\frac{(-0.7+ln(e^{0.1}+e^{0.2}+e^{0.7}+e^0+···))+(-0.9+ln(e^{0.05}+e^{0.05}+e^{0.9}+e^0+···))}{2}$

---

# 6. **反向传播和参数更新**

   - **反向传播**：`loss.backward()`，计算梯度。
   - **更新参数**：`optimizer.step()`，根据梯度更新模型参数。
   - **更新学习率**：`scheduler.step()`，调整学习率。

---

# 7. **计算准确率**

- **预测结果**：`torch.max(outputs, 2)`，获取每个字符的预测类别。
- **准确率计算**：只有当所有 4 个字符都预测正确时，才认为该样本正确。
累计准确率和损失。

> **注**：在 PyTorch 中，torch.max 是一个常用的函数，用于计算张量中的最大值及其索引。
> torch.max(outputs, 2) 的作用是从模型的输出中提取每个字符的预测类别。

**下面解释其过程：**
`outputs` 的形状为 `(batch_size, num_chars, num_classes)`，这里是`(32, 4, 36)`

> - `torch.max `函数有两个主要功能：1、计算最大值：找出每个位置的最大值。2、返回最大值的索引：返回最大值对应的索引。
> - `torch.max(outputs, 2)`：`outputs` 是模型的输出张量。`dim=2` 表示沿着最后一个维度（类别维度）计算最大值。
> - `torch.max` 返回两个张量：1、最大值张量：包含每个位置的最大值。2、索引张量：包含最大值对应的索引。

对于以下代码：
```py
_, predicted = torch.max(outputs, 2)
```

- `_` 是一个占位符，表示忽略最大值张量（因为我们不需要最大值，只需要最大值的索引）。

- `predicted` 是最大值的索引张量，表示模型对每个字符的预测类别。

- `predicted` 的形状：
  - `outputs` 的形状为 `(32, 4, 36)`，那么 `predicted` 的形状为 `(32, 4)`，表示每个样本的 4 个字符的预测类别索引。

# **示例**
假设 `outputs` 的一个样本（`batch_size=1`）如下：

```py
outputs = torch.tensor([[[0.1, 0.2, 0.7, 0.0],  # 第一个字符的概率分布
                         [0.8, 0.1, 0.05, 0.05],  # 第二个字符的概率分布
                         [0.01, 0.01, 0.95, 0.03],  # 第三个字符的概率分布
                         [0.2, 0.3, 0.4, 0.1]]])  # 第四个字符的概率分布
```

**执行：**`_, predicted = torch.max(outputs, 2)`

**结果：**`predicted = torch.tensor([[2, 0, 2, 2]])`，形状为 `(1, 4)`

**解释：** 使用`predicted[0]`即返回这个样本四个字符的预测索引，再根据`dataloader`定义的字典，由索引映射回字符即可.
>- 第一个字符的最大概率是 0.7，索引为 2。
>- 第二个字符的最大概率是 0.8，索引为 0。
>- 第三个字符的最大概率是 0.95，索引为 2。
>- 第四个字符的最大概率是 0.4，索引为 2。

---
# 8. **打印训练信息**

   - **训练损失**：`train_loss = total_loss / total`。
   - **训练准确率**：`train_acc = correct / total`。
   - **学习率**：`optimizer.param_groups[0]['lr']`。

---

# 9. **验证并保存模型**

   - **验证函数**：调用 `validate` 函数，计算验证集上的损失和准确率。
   - **保存条件**：如果验证准确率超过阈值（`save_threshold`），则保存模型。
   - **保存路径**：以验证准确率命名模型文件。

---

# 10.  **验证函数 (`validate`)**

     - **设置评估模式**：`model.eval()`，关闭 `Dropout` 和 `BatchNorm` 的训练行为。
     - **关闭梯度计算**：`torch.no_grad()`，减少内存占用，提高计算速度。
     - **验证过程**：与训练过程类似，但不进行反向传播和参数更新。
     - **返回结果**：返回验证集上的平均损失和准确率。



In [ ]:
# 计算参数量
model = TinyCNN()
print("模型参数量：", sum(p.numel() for p in model.parameters()))  # 约802.6KB

# 3. 训练配置
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = CaptchaDataset('path to your train data', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataset = CaptchaDataset('path to your validate data', transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=True)

# 启动训练
train(model,train_dataloader, val_dataloader, epochs=300, save_threshold=0.9985)

#把模型转为onnx模型
def convert_to_onnx(model, output_path):
    dummy_input = torch.randn(1, 3, 32, 90)
    torch.onnx.export(
        model, 
        dummy_input, 
        output_path,
        input_names=['input'], 
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 
                      'output': {0: 'batch_size'}}
    )


# **`transform`功能：**  

- 定义了数据预处理流程，包括：
- `transforms.ToTensor()`：将图像从 [0, 255] 的像素值范围转换为 [0.0, 1.0] 的浮点张量。
- `transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])`：对每个通道进行标准化，使数据分布更接近标准正态分布（均值为0，标准差为1）。

- **参数：**
  - `mean=[0.5, 0.5, 0.5]`：每个通道的均值。
  - `std=[0.5, 0.5, 0.5]`：每个通道的标准差。
  
---

# `convert_to_onnx(model, output_path)`  

**功能：**
- 将 PyTorch 模型转换为 ONNX 格式，便于在其他框架中使用（如 TensorRT、ONNX Runtime 等）。  

- `dummy_input`：模拟输入数据，用于推断模型结构。
  - 形状为 `(1, 3, 32, 90)`，表示：
    - 批量大小为 1。
    - 3 个通道（RGB）。
    - 图像高度为 32。
    - 图像宽度为 90。